snowball stemmer with pos, tf-idf vectorizer, normalization(minmax scaler), improved ML parameters

In [1]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

# Read the CSV files
train_sentiments = pd.read_csv("/kaggle/input/data-ml/train/train.csv")
test_sentiments = pd.read_csv("/kaggle/input/data-ml/test/test.csv")

# Text preprocessing function with stemming and POS tagging
def preprocess_review(review):
    tokens = word_tokenize(review)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Use SnowballStemmer for stemming
    stemmer = SnowballStemmer('english')
    tokens = [stemmer.stem(token) for token in tokens]

    # Perform POS tagging
    pos_tags = pos_tag(tokens)
    tokens = [word + '_' + pos for word, pos in pos_tags]

    return ' '.join(tokens)

train_sentiments['processed_reviews'] = train_sentiments['review'].apply(preprocess_review)
test_sentiments['processed_reviews'] = test_sentiments['review'].apply(preprocess_review)

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
train_tfidf = tfidf_vectorizer.fit_transform(train_sentiments['processed_reviews'])
test_tfidf = tfidf_vectorizer.transform(test_sentiments['processed_reviews'] )

# Perform Min-Max scaling for normalization
scaler = MinMaxScaler()
train_tfidf = scaler.fit_transform(train_tfidf.toarray())
test_tfidf= scaler.transform(test_tfidf.toarray())

# You can use train_tfidf_scaled and test_tfidf_scaled for further processing


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, BaggingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier

# Naive Bayes
nb_model = MultinomialNB(alpha=0.1) 
nb_model.fit(train_tfidf, train_sentiments['sentiment'])
nb_predictions = nb_model.predict(test_tfidf)
nb_accuracy = accuracy_score(test_sentiments['sentiment'], nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)
print("Classification Report for Naive Bayes:\n", classification_report(test_sentiments['sentiment'], nb_predictions))
print("\n")

# Random Forest
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42) 
rf_model.fit(train_tfidf, train_sentiments['sentiment'])
rf_predictions = rf_model.predict(test_tfidf)
rf_accuracy = accuracy_score(test_sentiments['sentiment'], rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)
print("Classification Report for Random Forest:\n", classification_report(test_sentiments['sentiment'], rf_predictions))
print("\n")

# Neural Network
nn_model =  MLPClassifier(hidden_layer_sizes=(128,), max_iter=200, alpha=0.0001, random_state=42)
nn_model.fit(train_tfidf, train_sentiments['sentiment'])
nn_predictions = nn_model.predict(test_tfidf)
nn_accuracy = accuracy_score(test_sentiments['sentiment'], nn_predictions)
print("Neural Network MLP CLASSIFIER Accuracy:", nn_accuracy)
print("Classification Report for Neural Network:\n", classification_report(test_sentiments['sentiment'], nn_predictions))
print("\n")

# Decision Tree
dt_model =  DecisionTreeClassifier(max_depth=10, min_samples_split=10, random_state=42)
dt_model.fit(train_tfidf, train_sentiments['sentiment'])
dt_predictions = dt_model.predict(test_tfidf)
dt_accuracy = accuracy_score(test_sentiments['sentiment'], dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)
print("Classification Report for Decision Tree:\n", classification_report(test_sentiments['sentiment'], dt_predictions))
print("\n")

# Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42)
gb_model.fit(train_tfidf, train_sentiments['sentiment'])
gb_predictions = gb_model.predict(test_tfidf)
gb_accuracy = accuracy_score(test_sentiments['sentiment'], gb_predictions)
print("Gradient Boosting Accuracy:", gb_accuracy)
print("Classification Report for Gradient Boosting:\n", classification_report(test_sentiments['sentiment'], gb_predictions))
print("\n")

# k-NN
knn_model = KNeighborsClassifier(n_neighbors=10)
knn_model.fit(train_tfidf, train_sentiments['sentiment'])
knn_predictions = knn_model.predict(test_tfidf)
knn_accuracy = accuracy_score(test_sentiments['sentiment'], knn_predictions)
print("k-NN Accuracy:", knn_accuracy)
print("Classification Report for k-NN:\n", classification_report(test_sentiments['sentiment'], knn_predictions))
print("\n")

# Stacking
stacking_model = StackingClassifier(
    estimators=[('nb', nb_model), ('rf', rf_model), ('knn', knn_model), ('dt', dt_model),('gb', gb_model)],
    final_estimator=GradientBoostingClassifier(n_estimators=100, random_state=42)
)
stacking_model.fit(train_tfidf, train_sentiments['sentiment'])
stacking_predictions = stacking_model.predict(test_tfidf)
stacking_accuracy = accuracy_score(test_sentiments['sentiment'], stacking_predictions)
print("Stacking Accuracy:", stacking_accuracy)
print("Classification Report for Stacking:\n", classification_report(test_sentiments['sentiment'], stacking_predictions))
print("\n")

2024-02-12 19:05:23.750646: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 19:05:23.750792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 19:05:23.754404: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Naive Bayes Accuracy: 0.8446
Classification Report for Naive Bayes:
               precision    recall  f1-score   support

    negative       0.85      0.84      0.84      9935
    positive       0.84      0.85      0.85     10065

    accuracy                           0.84     20000
   macro avg       0.84      0.84      0.84     20000
weighted avg       0.84      0.84      0.84     20000



Random Forest Accuracy: 0.8209
Classification Report for Random Forest:
               precision    recall  f1-score   support

    negative       0.85      0.78      0.81      9935
    positive       0.80      0.86      0.83     10065

    accuracy                           0.82     20000
   macro avg       0.82      0.82      0.82     20000
weighted avg       0.82      0.82      0.82     20000



Neural Network MLP CLASSIFIER Accuracy: 0.85665
Classification Report for Neural Network:
               precision    recall  f1-score   support

    negative       0.86      0.86      0.86      9935


In [ ]:
# Bagging
bagging_model = BaggingClassifier(estimator= ExtraTreesClassifier(), n_estimators=200, random_state=42)
bagging_model.fit(train_tfidf, train_sentiments['sentiment'])
bagging_predictions = bagging_model.predict(test_tfidf)
bagging_accuracy = accuracy_score(test_sentiments['sentiment'], bagging_predictions)
print("Bagging Accuracy:", bagging_accuracy)
print("Classification Report for Bagging:\n", classification_report(test_sentiments['sentiment'], bagging_predictions))
print("\n")

# Voting Classifier
voting_model = VotingClassifier(
    estimators=[('nb', nb_model), ('rf', rf_model), ('knn', knn_model), ('dt', dt_model), ('gb', gb_model), ('st', stacking_model)],
    voting='hard'
)
voting_model.fit(train_tfidf, train_sentiments['sentiment'])
voting_predictions = voting_model.predict(test_tfidf)
voting_accuracy = accuracy_score(test_sentiments['sentiment'], voting_predictions)
print("Voting Classifier Accuracy:", voting_accuracy)
print("Classification Report for Voting Classifier:\n", classification_report(test_sentiments['sentiment'], voting_predictions))

snowball stemmer with pos, tf-idf vectorizer, normalization( robust scaler), improved ML parameters

In [ ]:
import nltk
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

train_sentiments = pd.read_csv("/kaggle/input/data-ml/train/train.csv")
test_sentiments = pd.read_csv("/kaggle/input/data-ml/test/test.csv")

# Text preprocessing function with stemming and POS tagging
def preprocess_review(review):
    tokens = word_tokenize(review)
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]

    # Use SnowballStemmer for stemming
    stemmer = SnowballStemmer('english')
    tokens = [stemmer.stem(token) for token in tokens]

    # Perform POS tagging
    pos_tags = pos_tag(tokens)
    tokens = [word + '_' + pos for word, pos in pos_tags]

    return ' '.join(tokens)

train_sentiments['processed_reviews'] = train_sentiments['review'].apply(preprocess_review)
test_sentiments['processed_reviews'] = test_sentiments['review'].apply(preprocess_review)

# Vectorize the text data using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
train_tfidf = tfidf_vectorizer.fit_transform(train_sentiments['processed_reviews'])
test_tfidf = tfidf_vectorizer.transform(test_sentiments['processed_reviews'] )

# Perform robust scaling for normalization
scaler = RobustScaler()
train_tfidf = scaler.fit_transform(train_tfidf.toarray())
test_tfidf = scaler.transform(test_tfidf.toarray())

# You can use train_tfidf_scaled and test_tfidf_scaled for further processing

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import time
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, BaggingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier

start = time.time()
# Naive Bayes
nb_model = MultinomialNB(alpha=0.1) 
nb_model.fit(train_tfidf, train_sentiments['sentiment'])
nb_predictions = nb_model.predict(test_tfidf)
end = time.time()
nb_accuracy = accuracy_score(test_sentiments['sentiment'], nb_predictions)
print("Naive Bayes Accuracy:", nb_accuracy)
print("Classification Report for Naive Bayes:\n", classification_report(test_sentiments['sentiment'], nb_predictions))
print("\n")
print(start - end)


# Random Forest
rf_model = RandomForestClassifier(n_estimators=200, max_depth=10, random_state=42) 
rf_model.fit(train_tfidf, train_sentiments['sentiment'])
rf_predictions = rf_model.predict(test_tfidf)
rf_accuracy = accuracy_score(test_sentiments['sentiment'], rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)
print("Classification Report for Random Forest:\n", classification_report(test_sentiments['sentiment'], rf_predictions))
print("\n")

# Neural Network
nn_model =  MLPClassifier(hidden_layer_sizes=(128,), max_iter=200, alpha=0.0001, random_state=42)
nn_model.fit(train_tfidf, train_sentiments['sentiment'])
nn_predictions = nn_model.predict(test_tfidf)
nn_accuracy = accuracy_score(test_sentiments['sentiment'], nn_predictions)
print("Neural Network MLP CLASSIFIER Accuracy:", nn_accuracy)
print("Classification Report for Neural Network:\n", classification_report(test_sentiments['sentiment'], nn_predictions))
print("\n")

# Decision Tree
dt_model =  DecisionTreeClassifier(max_depth=10, min_samples_split=10, random_state=42)
dt_model.fit(train_tfidf, train_sentiments['sentiment'])
dt_predictions = dt_model.predict(test_tfidf)
dt_accuracy = accuracy_score(test_sentiments['sentiment'], dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)
print("Classification Report for Decision Tree:\n", classification_report(test_sentiments['sentiment'], dt_predictions))
print("\n")

# Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42)
gb_model.fit(train_tfidf, train_sentiments['sentiment'])
gb_predictions = gb_model.predict(test_tfidf)
gb_accuracy = accuracy_score(test_sentiments['sentiment'], gb_predictions)
print("Gradient Boosting Accuracy:", gb_accuracy)
print("Classification Report for Gradient Boosting:\n", classification_report(test_sentiments['sentiment'], gb_predictions))
print("\n")

# k-NN
knn_model = KNeighborsClassifier(n_neighbors=10)
knn_model.fit(train_tfidf, train_sentiments['sentiment'])
knn_predictions = knn_model.predict(test_tfidf)
knn_accuracy = accuracy_score(test_sentiments['sentiment'], knn_predictions)
print("k-NN Accuracy:", knn_accuracy)
print("Classification Report for k-NN:\n", classification_report(test_sentiments['sentiment'], knn_predictions))
print("\n")

# Stacking
stacking_model = StackingClassifier(
    estimators=[('nb', nb_model), ('rf', rf_model), ('knn', knn_model), ('dt', dt_model),('gb', gb_model)],
    final_estimator=GradientBoostingClassifier(n_estimators=100, random_state=42)
)
stacking_model.fit(train_tfidf, train_sentiments['sentiment'])
stacking_predictions = stacking_model.predict(test_tfidf)
stacking_accuracy = accuracy_score(test_sentiments['sentiment'], stacking_predictions)
print("Stacking Accuracy:", stacking_accuracy)
print("Classification Report for Stacking:\n", classification_report(test_sentiments['sentiment'], stacking_predictions))
print("\n")